In [1]:
import mbuild as mb
from openff.toolkit.topology import Molecule
import random
import numpy
import warnings
warnings.filterwarnings("ignore")

In [ ]:
thiophene = mb.load("c1ccsc1",smiles=True)
thiophene.save("thio.sdf",overwrite=True)

In [ ]:
#compound = Molecule.from_file("thio.sdf",file_format = "sdf")
#compound.visualize()

In [50]:
octet = {
        6 : 4,
        7 : 3,
        1 : 1,
        8 : 2,
        16 : 2}

def walk_molecule(atom, mol, visited, happy):
    visited[atom.molecule_atom_index]=True
    bonds = [b for b in mol.bonds if atom.molecule_atom_index in [b.atom1_index,b.atom2_index]]
    current_order = numpy.sum([int(b.bond_order) for b in bonds])
    print("im on {}, which is a {}".format(atom.molecule_atom_index,atom.atomic_number) )
    target = octet[atom.atomic_number]
    add_order = 0
    if current_order > target:
        print("\t too many electrons, decrementing the bond I came from")
        happy[atom.molecule_atom_index] = False
        add_order = -1
    if current_order == target:
        happy[atom.molecule_atom_index] = True
        print("\t and now I have the right number of bonds")
    if current_order < target:
        happy[atom.molecule_atom_index ] = False
        print("\t too few electrons, let's try increasing some bond order")
        add_order = 1   
    for b in bonds:
        next_atom = b.atom1
        if b.atom1_index == atom.molecule_atom_index:
            next_atom = b.atom2
    #this is creating a list with only the bond between atom and next atom
    inc_bond = [ib for ib in bonds if next_atom.molecule_atom_index in [b.atom1_index,b.atom2_index]]
    inc_bond[0].bond_order += add_order
    for b in bonds:
        next_atom = b.atom1
        if b.atom1_index == atom.molecule_atom_index:
            next_atom = b.atom2 
        if visited[next_atom.molecule_atom_index] == False:
            walk_molecule(mol.atoms[next_atom.molecule_atom_index],compound,visited,happy)
            """let's just increase the bond order of this 
            bond and try walking to the next atom, 
            aiming to come back if it's too high"""
    """inc_bond cant be in for loop or it adds 
    add_order onto the bondorder 3 times instead of once"""

            
compound = Molecule.from_file("thio.sdf",file_format = "sdf")
walk_molecule(compound.atoms[0],compound,[False]*len(compound.atoms),[False]*len(compound.atoms))

im on 0, which is a 6
	 too few electrons, let's try increasing some bond order
im on 1, which is a 6
	 and now I have the right number of bonds
im on 2, which is a 6
	 too few electrons, let's try increasing some bond order
im on 3, which is a 16
	 and now I have the right number of bonds
im on 4, which is a 6
	 too few electrons, let's try increasing some bond order
im on 8, which is a 1
	 and now I have the right number of bonds
im on 7, which is a 1
	 and now I have the right number of bonds
im on 6, which is a 1
	 and now I have the right number of bonds
im on 5, which is a 1
	 and now I have the right number of bonds


In [51]:
##check to see which bond orders increased and which indices belong to the bond. 
bond_list = [b for b in compound.bonds]
for b in bond_list:
    print(b.atom1_index,b.atom2_index)
    print(b.bond_order)

1 0
2
2 1
2
3 2
1
4 0
2
4 3
1
5 0
1
6 1
1
7 2
1
8 4
1


In [45]:
compound.visualize()

[12:16:42] Explicit valence for atom # 0 C, 5, is greater than permitted


AtomValenceException: Explicit valence for atom # 0 C, 5, is greater than permitted

In [ ]:
#bond_list = [b for b in compound.bonds if 1 not in [b.atom1.atomic_number, b.atom2.atomic_number]]
#atom_list = [a for a in compound.atoms if 1 not in [a.atomic_number]]
#def add_bonds(atom1_index, atom2_index, mol):
#    change_bonds1 = [b for b in mol.bonds if atom1_index in [b.atom1_index,b.atom2_index]]
#    change_bonds2 = [b for b in change_bonds1 if atom2_index in [b.atom1_index,b.atom2_index]]
#    change_bond = change_bonds2[0]
#    original_order = change_bond.bond_order
#    change_bond.bond_order = original_order + 1
#    print(change_bond.bond_order)
#add_bonds(1,2,compound)
#add_bonds(4,5,compound)
#add_bonds(6,7,compound)
#add_bonds(0,8,compound)